In [6]:
#1.Implement a perceptron from scratch 

import numpy as np
import os
import cv2

# Load training data
training_path = "/home/manichandana-sandhaboina/Downloads/train"
images = []
labels = []
for root, dirs, files in os.walk(training_path):
    for filename in files:
        filepath = os.path.join(root, filename)
        label = os.path.basename(root)
        image = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
        if image is not None:
            image = cv2.resize(image, (28, 28))
            images.append(image.flatten())
            labels.append(label)
X_train = np.array(images)
y_train = np.array(labels)

# Map labels to integers
label_to_int = {label: idx for idx, label in enumerate(np.unique(y_train))}
y_train = np.array([label_to_int[label] for label in y_train])

# Initialize perceptron
input_size = X_train.shape[1]
weights = np.zeros(input_size)
bias = 0
learning_rate = 0.01
epochs = 100

# Train perceptron
for epoch in range(epochs):
    for i in range(len(X_train)):
        prediction = np.dot(X_train[i], weights) + bias
        prediction = 1 if prediction >= 0 else 0
        if prediction != y_train[i]:
            update = learning_rate * (y_train[i] - prediction)
            weights += update * X_train[i]
            bias += update

# Evaluate training accuracy
correct = 0
for i in range(len(X_train)):
    prediction = np.dot(X_train[i], weights) + bias
    prediction = 1 if prediction >= 0 else 0
    if prediction == y_train[i]:
        correct += 1
train_acc = correct / len(X_train)
print(f"Training Accuracy: {train_acc:.2f}")

# Optionally, load validation data and evaluate accuracy
validation_path = "/home/manichandana-sandhaboina/Downloads/validation"
images = []
labels = []
for root, dirs, files in os.walk(validation_path):
    for filename in files:
        filepath = os.path.join(root, filename)
        label = os.path.basename(root)
        image = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
        if image is not None:
            image = cv2.resize(image, (28, 28))
            images.append(image.flatten())
            labels.append(label)
X_val = np.array(images)
y_val = np.array(labels)

label_to_int = {label: idx for idx, label in enumerate(np.unique(y_val))}
y_val = np.array([label_to_int[label] for label in y_val])

correct = 0
for i in range(len(X_val)):
    prediction = np.dot(X_val[i], weights) + bias
    prediction = 1 if prediction >= 0 else 0
    if prediction == y_val[i]:
        correct += 1
val_acc = correct / len(X_val)
print(f"Validation Accuracy: {val_acc:.2f}")

# Optionally, load testing data and evaluate accuracy
testing_path = "/home/manichandana-sandhaboina/Downloads/test"
images = []
labels = []
for root, dirs, files in os.walk(testing_path):
    for filename in files:
        filepath = os.path.join(root, filename)
        label = os.path.basename(root)
        image = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
        if image is not None:
            image = cv2.resize(image, (28, 28))
            images.append(image.flatten())
            labels.append(label)
X_test = np.array(images)
y_test = np.array(labels)

label_to_int = {label: idx for idx, label in enumerate(np.unique(y_test))}
y_test = np.array([label_to_int[label] for label in y_test])

correct = 0
for i in range(len(X_test)):
    prediction = np.dot(X_test[i], weights) + bias
    prediction = 1 if prediction >= 0 else 0
    if prediction == y_test[i]:
        correct += 1
test_acc = correct / len(X_test)
print(f"Testing Accuracy: {test_acc:.2f}")



Training Accuracy: 1.00
Validation Accuracy: 0.91
Testing Accuracy: 1.00


In [2]:
#2.Build and train a simple neural network using a framework like TensorFlow or PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from PIL import Image
import os

transform = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.ToTensor()
])

training_path = "/home/manichandana-sandhaboina/Downloads/train"
validation_path = "/home/manichandana-sandhaboina/Downloads/validation"
testing_path = "/home/manichandana-sandhaboina/Downloads/test"

root = training_path
transform = transform
samples = []
for root, _, fnames in sorted(os.walk(root)):
    for fname in sorted(fnames):
        path = os.path.join(root, fname)
        try:
            img = Image.open(path).convert('L')
            samples.append((path, img))
        except Exception as e:
            print(f"Skipping {path}: {str(e)}")

train_loader = DataLoader([(transform(sample[1]), 0) for sample in samples], batch_size=32, shuffle=True)

root = validation_path
transform = transform
samples = []
for root, _, fnames in sorted(os.walk(root)):
    for fname in sorted(fnames):
        path = os.path.join(root, fname)
        try:
            img = Image.open(path).convert('L')
            samples.append((path, img))
        except Exception as e:
            print(f"Skipping {path}: {str(e)}")

val_loader = DataLoader([(transform(sample[1]), 0) for sample in samples], batch_size=32, shuffle=False)

root = testing_path
transform = transform
samples = []
for root, _, fnames in sorted(os.walk(root)):
    for fname in sorted(fnames):
        path = os.path.join(root, fname)
        try:
            img = Image.open(path).convert('L')
            samples.append((path, img))
        except Exception as e:
            print(f"Skipping {path}: {str(e)}")

test_loader = DataLoader([(transform(sample[1]), 0) for sample in samples], batch_size=32, shuffle=False)

model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(28 * 28, 128),
    nn.ReLU(),
    nn.Linear(128, 10)
)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(3):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)

    # Calculate training accuracy
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for inputs, labels in train_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    train_acc = correct / total

    # Calculate validation accuracy
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    val_acc = correct / total

    print(f"Epoch [{epoch+1}/10], Loss: {epoch_loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")

# Evaluate the model on the test set
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
test_acc = correct / total
print(f"Testing Accuracy: {test_acc:.4f}")




Skipping /home/manichandana-sandhaboina/Downloads/train/MNIST/raw/t10k-images-idx3-ubyte: cannot identify image file '/home/manichandana-sandhaboina/Downloads/train/MNIST/raw/t10k-images-idx3-ubyte'
Skipping /home/manichandana-sandhaboina/Downloads/train/MNIST/raw/t10k-images-idx3-ubyte.gz: cannot identify image file '/home/manichandana-sandhaboina/Downloads/train/MNIST/raw/t10k-images-idx3-ubyte.gz'
Skipping /home/manichandana-sandhaboina/Downloads/train/MNIST/raw/t10k-labels-idx1-ubyte: cannot identify image file '/home/manichandana-sandhaboina/Downloads/train/MNIST/raw/t10k-labels-idx1-ubyte'
Skipping /home/manichandana-sandhaboina/Downloads/train/MNIST/raw/t10k-labels-idx1-ubyte.gz: cannot identify image file '/home/manichandana-sandhaboina/Downloads/train/MNIST/raw/t10k-labels-idx1-ubyte.gz'
Skipping /home/manichandana-sandhaboina/Downloads/train/MNIST/raw/train-images-idx3-ubyte: cannot identify image file '/home/manichandana-sandhaboina/Downloads/train/MNIST/raw/train-images-idx

In [2]:
#3.Create a multi-layer perceptron (MLP) for digit classification (MNIST dataset)
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

# Define transforms for preprocessing
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load data from folders
train_data = datasets.MNIST("/home/manichandana-sandhaboina/Downloads/train", download=True, train=True, transform=transform)
validation_data = datasets.MNIST("/home/manichandana-sandhaboina/Downloads/validation", download=True, train=False, transform=transform)
test_data = datasets.MNIST("/home/manichandana-sandhaboina/Downloads/test", download=True, train=False, transform=transform)

# Data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False)
validation_loader = torch.utils.data.DataLoader(validation_data, batch_size=64, shuffle=False)

# Multi-layer perceptron (MLP) model
mlp = nn.Sequential(
    nn.Linear(784, 128),
    nn.ReLU(),
    nn.Linear(128, 64),
    nn.ReLU(),
    nn.Linear(64, 10)
)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(mlp.parameters(), lr=0.01, momentum=0.9)

# Train the network
for epoch in range(20):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs = inputs.view(-1, 784)
        optimizer.zero_grad()
        outputs = mlp(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {running_loss/i}')

# Evaluate on training set
correct = 0
total = 0
with torch.no_grad():
    for data in train_loader:
        inputs, labels = data
        inputs = inputs.view(-1, 784)
        outputs = mlp(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f'Training Accuracy: {100 * correct / total} %')

# Evaluate on test set
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        inputs, labels = data
        inputs = inputs.view(-1, 784)
        outputs = mlp(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f'Test Accuracy: {100 * correct / total} %')

# Evaluate on validation set
correct = 0
total = 0
with torch.no_grad():
    for data in validation_loader:
        inputs, labels = data
        inputs = inputs.view(-1, 784)
        outputs = mlp(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f'Validation Accuracy: {100 * correct / total} %')


Epoch 1, Loss: 0.43349777888844593
Epoch 2, Loss: 0.17687799556333902
Epoch 3, Loss: 0.12952829222319093
Epoch 4, Loss: 0.10619127004344728
Epoch 5, Loss: 0.08731912212991011
Epoch 6, Loss: 0.07421592731707657
Epoch 7, Loss: 0.06527332497736799
Epoch 8, Loss: 0.05884556728986707
Epoch 9, Loss: 0.05234267381593891
Epoch 10, Loss: 0.04615782773017331
Epoch 11, Loss: 0.04149583718455432
Epoch 12, Loss: 0.03548893290002837
Epoch 13, Loss: 0.033439223111193156
Epoch 14, Loss: 0.029517166675539316
Epoch 15, Loss: 0.025470443684500896
Epoch 16, Loss: 0.02378686530941828
Epoch 17, Loss: 0.025655421347896484
Epoch 18, Loss: 0.020200532132992274
Epoch 19, Loss: 0.020018791839263722
Epoch 20, Loss: 0.018038608176792467
Training Accuracy: 99.14333333333333 %
Test Accuracy: 97.67 %
Validation Accuracy: 97.67 %


In [3]:
#4Experiment with different regularization techniques on a neural network
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from PIL import Image
from torch.utils.data import DataLoader, TensorDataset
import os

# Define paths to datasets
training_path = "/home/manichandana-sandhaboina/Downloads/train"
validation_path = "/home/manichandana-sandhaboina/Downloads/validation"
testing_path = "/home/manichandana-sandhaboina/Downloads/test"

# Define transforms for preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Paths to datasets
datasets = [
    {'name': 'train', 'path': training_path},
    {'name': 'validation', 'path': validation_path},
    {'name': 'test', 'path': testing_path}
]

# Initialize datasets and dataloaders
data_loaders = {}
for dataset in datasets:
    images = [f for f in os.listdir(dataset['path']) if f.endswith('.jpg') or f.endswith('.png')]
    data = []
    labels = []
    for img_name in images:
        img_path = os.path.join(dataset['path'], img_name)
        image = Image.open(img_path)
        image = transform(image)
        data.append(image)
        labels.append(torch.tensor(0))  # Assuming all labels are 0 for simplicity

    # Convert lists to tensors
    data_tensor = torch.stack(data)
    labels_tensor = torch.tensor(labels)

    # Create TensorDataset and DataLoader
    data_loaders[dataset['name']] = DataLoader(TensorDataset(data_tensor, labels_tensor), batch_size=64, shuffle=dataset['name'] == 'train')

# Neural network model
net = nn.Sequential(
    nn.Linear(224 * 224 * 3, 128),
    nn.ReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(128, 64),
    nn.ReLU(),
    nn.Linear(64, 10)
)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()

# Regularization techniques
regularization_techniques = ['L1', 'L2', 'Dropout']

for technique in regularization_techniques:
    print(f"Training with {technique} regularization...")
    
    # Reset model parameters and optimizer
    for module in net.modules():
        if isinstance(module, nn.Linear):
            nn.init.xavier_uniform_(module.weight)
            nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Dropout):
            module.p = 0.5
            
    optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

    # Train the network
    num_epochs = 10 # Increase number of epochs
    for epoch in range(num_epochs):
        running_loss = 0.0
        net.train()  # Set the model to train mode
        for dataset_name in ['train']:
            for i, (inputs, labels) in enumerate(data_loaders[dataset_name], 0):
                optimizer.zero_grad()
                inputs = inputs.view(-1, 224 * 224 * 3)
                outputs = net(inputs)
                loss = criterion(outputs, labels)
                
                # Apply regularization for L1 and L2
                if technique == 'L1' or technique == 'L2':
                    l1_lambda = 0.00001  # Adjust regularization strength
                    l2_lambda = 0.0001   # Adjust regularization strength
                    l1_reg = torch.tensor(0., requires_grad=False)
                    l2_reg = torch.tensor(0., requires_grad=False)
                    for param in net.parameters():
                        l1_reg += torch.norm(param, 1)
                        l2_reg += torch.norm(param, 2)
                    loss += l1_lambda * l1_reg + l2_lambda * l2_reg
                
                loss.backward()
                optimizer.step()
                running_loss += loss.item()

        # Calculate training accuracy
        net.eval()  # Set the model to evaluation mode
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in data_loaders['train']:
                inputs = inputs.view(-1, 224 * 224 * 3)
                outputs = net(inputs)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
            train_accuracy = 100 * correct / total

            # Calculate validation accuracy
            correct = 0
            total = 0
            for inputs, labels in data_loaders['validation']:
                inputs = inputs.view(-1, 224 * 224 * 3)
                outputs = net(inputs)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
            validation_accuracy = 100 * correct / total

            print(f'Epoch {epoch+1}, Loss: {running_loss / len(data_loaders["train"])}, Train Accuracy: {train_accuracy}%, Validation Accuracy: {validation_accuracy}%')

    # Evaluate on test set
    correct = 0
    total = 0
    net.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        for inputs, labels in data_loaders['test']:
            inputs = inputs.view(-1, 224 * 224 * 3)
            outputs = net(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        test_accuracy = 100 * correct / total
    print(f'Test Accuracy with {technique} regularization: {test_accuracy}%')

Training with L1 regularization...
Epoch 1, Loss: 2.4368600845336914, Train Accuracy: 100.0%, Validation Accuracy: 100.0%
Epoch 2, Loss: 0.6417902708053589, Train Accuracy: 100.0%, Validation Accuracy: 100.0%
Epoch 3, Loss: 0.6720300912857056, Train Accuracy: 100.0%, Validation Accuracy: 100.0%
Epoch 4, Loss: 0.7013783057530721, Train Accuracy: 100.0%, Validation Accuracy: 100.0%
Epoch 5, Loss: 0.7260257403055826, Train Accuracy: 100.0%, Validation Accuracy: 100.0%
Epoch 6, Loss: 0.745365560054779, Train Accuracy: 100.0%, Validation Accuracy: 100.0%
Epoch 7, Loss: 0.7600256204605103, Train Accuracy: 100.0%, Validation Accuracy: 100.0%
Epoch 8, Loss: 0.7708768844604492, Train Accuracy: 100.0%, Validation Accuracy: 100.0%
Epoch 9, Loss: 0.7788434227307638, Train Accuracy: 100.0%, Validation Accuracy: 100.0%
Epoch 10, Loss: 0.7845848401387533, Train Accuracy: 100.0%, Validation Accuracy: 100.0%
Test Accuracy with L1 regularization: 100.0%
Training with L2 regularization...
Epoch 1, Loss: 

In [5]:
#5.Compare performance with various optimization algorithms
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from PIL import Image
from torch.utils.data import DataLoader, TensorDataset
import os

# Define paths to datasets
training_path = "/home/manichandana-sandhaboina/Downloads/train"
validation_path = "/home/manichandana-sandhaboina/Downloads/validation"
testing_path = "/home/manichandana-sandhaboina/Downloads/test"

# Define transforms for preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Paths to datasets
datasets = [
    {'name': 'train', 'path': training_path},
    {'name': 'validation', 'path': validation_path},
    {'name': 'test', 'path': testing_path}
]

# Initialize datasets and dataloaders
data_loaders = {}
for dataset in datasets:
    images = [f for f in os.listdir(dataset['path']) if f.endswith('.jpg') or f.endswith('.png')]
    data = []
    labels = []
    for img_name in images:
        img_path = os.path.join(dataset['path'], img_name)
        image = Image.open(img_path)
        image = transform(image)
        data.append(image)
        labels.append(torch.tensor(0))  # Assuming all labels are 0 for simplicity

    # Convert lists to tensors
    data_tensor = torch.stack(data)
    labels_tensor = torch.tensor(labels)

    # Create TensorDataset and DataLoader
    data_loaders[dataset['name']] = DataLoader(TensorDataset(data_tensor, labels_tensor), batch_size=64, shuffle=dataset['name'] == 'train')

# Neural network model
net = nn.Sequential(
    nn.Linear(224 * 224 * 3, 128),
    nn.ReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(128, 64),
    nn.ReLU(),
    nn.Linear(64, 10)
)

# Loss function
criterion = nn.CrossEntropyLoss()

# List of optimization algorithms to compare
optimizers = [
    {'name': 'SGD', 'optimizer': optim.SGD(net.parameters(), lr=0.01, momentum=0.9)},
    {'name': 'Adam', 'optimizer': optim.Adam(net.parameters(), lr=0.001)},
    {'name': 'RMSprop', 'optimizer': optim.RMSprop(net.parameters(), lr=0.001)},
    {'name': 'Adagrad', 'optimizer': optim.Adagrad(net.parameters(), lr=0.01)},
    {'name': 'Adadelta', 'optimizer': optim.Adadelta(net.parameters(), lr=1.0)}
]

# Training loop
for optimizer_info in optimizers:
    optimizer_name = optimizer_info['name']
    optimizer = optimizer_info['optimizer']
    print(f"Training with {optimizer_name} optimizer...")

    # Reset model parameters
    for module in net.modules():
        if isinstance(module, nn.Linear):
            nn.init.xavier_uniform_(module.weight)
            nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Dropout):
            module.p = 0.5

    # Train the network
    num_epochs = 10
    for epoch in range(num_epochs):
        running_loss = 0.0
        net.train()  # Set the model to train mode
        for dataset_name in ['train']:
            for i, (inputs, labels) in enumerate(data_loaders[dataset_name], 0):
                optimizer.zero_grad()
                inputs = inputs.view(-1, 224 * 224 * 3)
                outputs = net(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()

        # Calculate training accuracy
        net.eval()  # Set the model to evaluation mode
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in data_loaders['train']:
                inputs = inputs.view(-1, 224 * 224 * 3)
                outputs = net(inputs)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
            train_accuracy = 100 * correct / total

            # Calculate validation accuracy
            correct = 0
            total = 0
            for inputs, labels in data_loaders['validation']:
                inputs = inputs.view(-1, 224 * 224 * 3)
                outputs = net(inputs)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
            validation_accuracy = 100 * correct / total

            print(f'Epoch {epoch+1}, Loss: {running_loss / len(data_loaders["train"])}, Train Accuracy: {train_accuracy}%, Validation Accuracy: {validation_accuracy}%')

    # Evaluate on test set
    correct = 0
    total = 0
    net.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        for inputs, labels in data_loaders['test']:
            inputs = inputs.view(-1, 224 * 224 * 3)
            outputs = net(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        test_accuracy = 100 * correct / total
    print(f'Test Accuracy with {optimizer_name} optimizer: {test_accuracy}%')


Training with SGD optimizer...
Epoch 1, Loss: 0.8833242257436117, Train Accuracy: 100.0%, Validation Accuracy: 100.0%
Epoch 2, Loss: 0.0, Train Accuracy: 100.0%, Validation Accuracy: 100.0%
Epoch 3, Loss: 0.0, Train Accuracy: 100.0%, Validation Accuracy: 100.0%
Epoch 4, Loss: 0.0, Train Accuracy: 100.0%, Validation Accuracy: 100.0%
Epoch 5, Loss: 0.0, Train Accuracy: 100.0%, Validation Accuracy: 100.0%
Epoch 6, Loss: 0.0, Train Accuracy: 100.0%, Validation Accuracy: 100.0%
Epoch 7, Loss: 0.0, Train Accuracy: 100.0%, Validation Accuracy: 100.0%
Epoch 8, Loss: 0.0, Train Accuracy: 100.0%, Validation Accuracy: 100.0%
Epoch 9, Loss: 0.0, Train Accuracy: 100.0%, Validation Accuracy: 100.0%
Epoch 10, Loss: 0.0, Train Accuracy: 100.0%, Validation Accuracy: 100.0%
Test Accuracy with SGD optimizer: 100.0%
Training with Adam optimizer...
Epoch 1, Loss: 6.733063141504924, Train Accuracy: 100.0%, Validation Accuracy: 100.0%
Epoch 2, Loss: 0.2000760038693746, Train Accuracy: 100.0%, Validation Accu